# Notebook for basic movement of Mabu's head

In [21]:
!pip uninstall serial -y

In [2]:
!pip install pyserial
!pip install ipywidgets

Defaulting to user installation because normal site-packages is not writeable
  Using cached pyserial-3.5-py2.py3-none-any.whl.metadata (1.6 kB)
Using cached pyserial-3.5-py2.py3-none-any.whl (90 kB)



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: C:\Users\nicol\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: C:\Users\nicol\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
import serial
import serial.tools.list_ports
import time

In [4]:
# Get a list of all available serial ports
ports = serial.tools.list_ports.comports()

# Print information about each port
if not ports:
    print("No serial ports found.")
else:
    print("Available serial ports:")
    for port in ports:
        print(f"- {port.device}")
        print(f"  Description: {port.description}")
        print(f"  Hardware ID: {port.hwid}")
        print()

Available serial ports:
- COM16
  Description: USB-SERIAL CH340 (COM16)
  Hardware ID: USB VID:PID=1A86:7523 SER= LOCATION=1-3

- COM4
  Description: Standard Serial over Bluetooth link (COM4)
  Hardware ID: BTHENUM\{00001101-0000-1000-8000-00805F9B34FB}_LOCALMFG&0002\7&17EE80CD&0&B46C4757ADF4_C00000000

- COM5
  Description: Standard Serial over Bluetooth link (COM5)
  Hardware ID: BTHENUM\{00001101-0000-1000-8000-00805F9B34FB}_LOCALMFG&0000\7&17EE80CD&0&000000000000_00000000



In [5]:
# Initialize serial connection
ser = serial.Serial('COM16', 57600, timeout=1) 

In [9]:
def fletcher8(data):
    sum1 = 0
    sum2 = 0
    for byte in data:
        sum1 = (sum1 + byte) % 255
        sum2 = (sum2 + sum1) % 255
    return (sum2 << 8) | sum1

def move_robot_part(part, value):
    # Map parts to their respective command codes
    part_codes = {
        'LDL': 0x40,
        'LDR': 0x20,
        'ELR': 0x10,
        'EUD': 0x08,
        'NE': 0x04,
        'NR': 0x02,
        'NT': 0x01
    }

    if part not in part_codes:
        raise ValueError("Invalid part specified")

    # Map 0-100 to 0-255
    mapped_value = int(value * 2.55)

    # Construct the command
    command = bytearray([0xfa, 0x00, 0x04, 0x01, part_codes[part], 0x01, mapped_value])

    # Calculate checksum usign fletcher8 algorithm
    checksum = fletcher8(command)
    command.append(checksum >> 8)
    command.append(checksum & 0xff)

    # Send the command
    ser.write(command)
    #time.sleep(0.1)  # Small delay to ensure command is processed

# Initialize serial connection
# ser = serial.Serial('COM19', 57600, timeout=1)

def move_all_parts(ldl, ldr, elr, eud, ne, nr, nt):
    move_robot_part('LDL', ldl)
    move_robot_part('LDR', ldr)
    move_robot_part('ELR', elr)
    move_robot_part('EUD', eud)
    move_robot_part('NE', ne)
    move_robot_part('NR', nr)
    move_robot_part('NT', nt)



In [7]:
# Example usage:
move_all_parts(50, 50, 50, 50, 50, 50, 50)

In [8]:
from ipywidgets import interactive, FloatSlider
import ipywidgets as widgets

def update_robot(LDL, LDR, ELR, EUD, NE, NR, NT):
    move_robot_part('LDL', LDL)
    move_robot_part('LDR', LDR)
    move_robot_part('ELR', ELR)
    move_robot_part('EUD', EUD)
    move_robot_part('NE', NE)
    move_robot_part('NR', NR)
    move_robot_part('NT', NT)

sliders = [
    FloatSlider(min=0, max=100, step=1, description=part)
    for part in ['LDL', 'LDR', 'ELR', 'EUD', 'NE', 'NR', 'NT']
]

interactive_plot = interactive(update_robot, LDL=sliders[0], LDR=sliders[1],
                               ELR=sliders[2], EUD=sliders[3], NE=sliders[4],
                               NR=sliders[5], NT=sliders[6])

display(interactive_plot)

interactive(children=(FloatSlider(value=0.0, description='LDL', step=1.0), FloatSlider(value=0.0, description=…

In [ ]:
ser.close()